In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from gettsim import (
    compute_taxes_and_transfers,
    create_synthetic_data,
    set_up_policy_environment,
)

# Präsenzphase: Kinderzuschlag und Wohngeld

In diesem Notebook beschäftigen wir uns damit, wie der Kinderzuschlag und Wohngeld sich
auf die finanzielle Lage von Geringverdienern auswirkt. Anstelle eines Vergleichs
unterschiedlicher Haushaltstypen vergleichen wir diesmal, wie der Kinderzuschlag sich
auf bestimmte Haushaltstypen auswirkt.

Wir werden dies für zwei verschiedene Haushaltstypen machen:

- 2 Erwachsene, 1 Kind
- 2 Erwachsene, 2 Kinder

Dafür müssen wir, wie immer, zunächst einen entsprechenden Datensatz erstellen.

## Erstellung der Daten (10 Minuten)

*Achten sie heute im besonderen Maße auf die Wohnkosten, da diese hochrelevant für den Kinderzuschlag sind!* 

In [ ]:
min_einkommen = 500
max_einkommen = 5500
anzahl_schritte = 501

In [ ]:
# Zwei Erwachsene, ein Kind
df_2e_1k = create_synthetic_data(
    n_adults=2,
    n_children=1,
    specs_heterogeneous={
        "bruttolohn_m": [
            [i / 2, i / 2, 0]
            for i in np.linspace(min_einkommen, max_einkommen, anzahl_schritte)
        ],
    },
)

In [ ]:
# Zwei Erwachsene, zwei Kinder
df_2e_2k = create_synthetic_data(
    n_adults=2,
    n_children=2,
    specs_heterogeneous={
        "bruttolohn_m": [
            [i / 2, i / 2, 0, 0]
            for i in np.linspace(min_einkommen, max_einkommen, anzahl_schritte)
        ],
    },
)

for var in ["hh_id", "tu_id", "p_id"]:
    df_2e_2k[var] += 1 + df_2e_1k[var].max()

Nun erstellen wir einen großen Datensatz, der die eben erstellten enthält.

In [ ]:
inputs_personen = pd.concat(
    [df_2e_1k, df_2e_2k],
    keys=[
        "2 Erwachsene, 1 Kind",
        "2 Erwachsene, 2 Kinder",
    ],
    names=["Haushaltstyp", "lfd_nr"],
)
inputs_personen

Die Berechnung des zu versteuernden Einkommens vor Berücksichtigung des Kinderfreibetrags kürzen wir wieder ab.

In [ ]:
inputs_personen["_zu_verst_eink_ohne_kinderfreib_tu"] = (
    inputs_personen.groupby("tu_id")["bruttolohn_m"].transform("sum") * 12
)

Miete und Heizkosten hängen im automatisch erstellten Datensatz von der Haushaltsgröße ab:

In [ ]:
inputs_personen[["heizkosten_m_hh", "bruttokaltmiete_m_hh"]]

Wir setzen stattdessen die gleichen Werte für beide Haushaltstypen und nehmen etwas höhere sowie runde Zahlen.

In [ ]:
inputs_personen["heizkosten_m_hh"] = 100.0
inputs_personen["bruttokaltmiete_m_hh"] = 700.0

## Berechnung der Transfers und Steuern (20 Minuten)

Wir setzen zunächst wieder die Parameter des Steuer- und Transfersystems auf die des Jahres 2023 fest.

In [ ]:
params_dict, policy_func_dict = set_up_policy_environment(2023)

### Nutzen des "vor_vorrang" Targets von GETTSIM:

In den letzten Wochen haben wir bereits das `arbeitsl_geld_2_vor_vorrang_m_hh` Target in
die GETTSIM Funktion eingegeben, jedoch ohne wirklich darauf einzugehen. Heute wird der
Unterschied zwischen `arbeitsl_geld_2_vor_vorrang_m_hh` und dem "eigentlichen"
Bürgergeld-Target `arbeitsl_geld_2_m_hh` deutlich.

- `arbeitsl_geld_2_m_hh`: Bürgergeld ist eine subsidiäre Leistung. Das heißt, wenn der
  Bedarf einer Familie ohne Bürgergeld gedeckt werden kann, erlischt der Anspruch. Für
  GETTSIM bedeutet dies, dass sobald die Summe aus (Erwerbs-)einkommen, Kinderzuschlag und
  Wohngeld mindestens so hoch ist wie der Anspruch auf Bürgergeld (zzgl. des
  (Erwerbs-)einkommens), der Zahlbetrag des Bürgergelds auf Null gesetzt wird.

- `arbeitsl_geld_2_vor_vorrang_m_hh`: Bei diesem Target ist die automatische
  *Vorrangprüfung* außer Kraft gesetzt. Dadurch gibt GETTSIM den Bürgergeldanspruch aus,
  unabhängig davon, ob andere Sozialleistungen vorrangig wären. Das erlaubte uns in den
  letzten Wochen, die Mechanik des Bürgergelds zu verstehen, ohne auf Kinderzuschlag und
  Wohngeld einzugehen.

Drei Anmerkungen:

1. Da wir weiter mit `arbeitsl_geld_2_vor_vorrang_m_hh` arbeiten, können wir das
   verfügbare Einkommen jetzt nicht mehr als die Summe aller Nettoeinkünfte und
   Transfers berechnen, da ein gleichzeitiger Bezug von Bürgergeld und
   Wohngeld/Kinderzuschlag ausgeschlossen ist.
2. Im unteren Einkommensbereich ist es die Regel, dass Kinderzuschlag und Wohngeld den
   Bedarf der Familie nicht decken, hinzu kommen in beiden Fällen
   Mindesteinkommensgrenzen. Da die `wohngeld_m_hh` und `kinderzuschl_m_hh` analog zu 
   `arbeitsl_geld_2_m_hh` funktionieren (Anspruch und Vorrang also schon geprüft sind), 
   sind diese bereits Null. Wir ignorieren diese Einkommensbereiche.
3. Als letzten Schritt vor Aufgabe 1 sehen Sie Graphiken, die Wohngeld und
   Kinderzuschlag gegen das Bruttoeinkommen abtragen. Sie brauchen diese nur als
   Hintergrund für die folgende Aufgabe, bitte machen Sie sich zunächst über diese
   Graphiken keine großen Gedanken.

In [ ]:
ergebnisse_personen = compute_taxes_and_transfers(
    data=inputs_personen,
    functions=policy_func_dict,
    params=params_dict,
    columns_overriding_functions=[
        "_zu_verst_eink_ohne_kinderfreib_tu",
    ],
    targets=[
        "kindergeld_m_hh",
        "eink_st_tu",
        "soli_st_tu",
        "sozialv_beitr_m",
        "wohngeld_m_hh",
        "kinderzuschl_m_hh",
        "arbeitsl_geld_2_vor_vorrang_m_hh",
    ],
)

# Mithilfe der p_id wieder hh_id, bruttolohn_m an das DataFrame joinen.
ergebnisse_personen = ergebnisse_personen.join(
    inputs_personen.reset_index().set_index("p_id")[["hh_id", "bruttolohn_m"]],
)

In [ ]:
# Zahlen auf Haushaltsebene aggregieren: Summieren der Größen auf Individuenebene
# und Übernehmen der (max.) Größen auf Haushalts- und Tax-unit-Ebene.
ergebnisse = ergebnisse_personen.groupby("hh_id").agg(
    {
        "bruttolohn_m": lambda x: x.sum(),
        "kindergeld_m_hh": lambda x: x.max(),
        "eink_st_tu": lambda x: x.max() / 12,
        "soli_st_tu": lambda x: x.max() / 12,
        "sozialv_beitr_m": lambda x: x.sum(),
        "wohngeld_m_hh": lambda x: x.max(),
        "kinderzuschl_m_hh": lambda x: x.max(),
        "arbeitsl_geld_2_vor_vorrang_m_hh": lambda x: x.max(),
    },
)

# Kenntlich machen, dass es sich jetzt um Haushaltsgrößen handelt.
ergebnisse = ergebnisse.rename(
    columns={
        "bruttolohn_m": "bruttolohn_m_hh",
        "sozialv_beitr_m": "sozialv_beitr_m_hh",
    },
)

# Mithilfe der hh_id den Haushaltstyp and das DataFrame joinen.
ergebnisse = ergebnisse.join(
    inputs_personen.reset_index().set_index("hh_id")["Haushaltstyp"],
).drop_duplicates()

# Index für gute Übersicht setzen.
ergebnisse = ergebnisse.set_index(["Haushaltstyp", "bruttolohn_m_hh"])
ergebnisse

In [ ]:
px.line(
    ergebnisse.reset_index(),
    x="bruttolohn_m_hh",
    y="wohngeld_m_hh",
    color="Haushaltstyp",
    title="Wohngeld",
)

In [ ]:
px.line(
    ergebnisse.reset_index(),
    x="bruttolohn_m_hh",
    y="kinderzuschl_m_hh",
    color="Haushaltstyp",
    title="Kinderzuschlag",
)

## Aufgabe 1: Berechnung des verfügbaren Einkommens (25 Minuten)

Wie Sie in der Selbstlernphase gelernt haben, schließen sich der Bezug von Bürgergeld
sowie Wohngeld und/oder Kinderzuschlag gegenseitig aus. In dieser Aufgabe geht es darum,
zu verdeutlichen, wie der Vorrang von Wohngeld/Kinderzuschlag über Einkommensgruppen und
Haushaltstypen entwickelt.

Berechnen sie das verfügbare Einkommen für jeden Haushaltstyp gegeben der folgenden drei Fälle:

1. Der Haushalt beantragt (und bezieht) Kinderzuschlag und Wohngeld, falls möglich, aber nie Bürgergeld.

2. Der Haushalt beantragt (und bezieht) Bürgergeld, falls möglich, aber nie Kinderzuschlag oder Wohngeld.

3. Der Haushalt wählt die Transfers, die am vorteilhaftesten sind.


Berechnen Sie zuerst die **Transfers** in jedem Szenario, in dem Sie für jedes Szenario eine **eigene Spalte anfertigen** (z.B. `transfers_kiz_und_wg`, `transfers_bürgergeld`, `transfers_vorrang`). 

Danach sind Sie bereit, das **verfügbare Einkommen** für jedes Szenario zu berechenen.

In [ ]:
## solution
# Transfers
ergebnisse["transfers_kiz_und_wg"] = ergebnisse[
    ["kinderzuschl_m_hh", "wohngeld_m_hh", "kindergeld_m_hh"]
].sum(axis=1)
ergebnisse["transfers_bürgergeld"] = ergebnisse[
    ["arbeitsl_geld_2_vor_vorrang_m_hh", "kindergeld_m_hh"]
].sum(axis=1)
ergebnisse["transfers_vorrang"] = ergebnisse[
    ["transfers_kiz_und_wg", "transfers_bürgergeld"]
].max(axis=1)

In [ ]:
## solution
ergebnisse = ergebnisse.reset_index()
for ext in "_kiz_und_wg", "_bürgergeld", "_vorrang":
    name = "verfügbares_einkommen" + ext
    ergebnisse[name] = (
        ergebnisse["bruttolohn_m_hh"]
        + ergebnisse["transfers" + ext]
        - ergebnisse["eink_st_tu"]
        - ergebnisse["soli_st_tu"]
        - ergebnisse["sozialv_beitr_m_hh"]
    )
ergebnisse = ergebnisse.set_index(["Haushaltstyp", "bruttolohn_m_hh"])
ergebnisse

## Aufgabe 2: Erstellung von Grafiken (14 Minuten)

Erstellen Sie zwei Grafiken (eine für jeden Haushaltstyp) in dem Sie die Abhängigkeit des verfügbaren Einkommens vom monatlichen Bruttolohn in den unterschiedlichen Szenarien illustrieren.

In [ ]:
## solution
px.line(
    ergebnisse.reset_index().query("Haushaltstyp == '2 Erwachsene, 1 Kind'"),
    x="bruttolohn_m_hh",
    y=[
        "verfügbares_einkommen_kiz_und_wg",
        "verfügbares_einkommen_bürgergeld",
        "verfügbares_einkommen_vorrang",
    ],
    title="2 Erwachsene, 1 Kind",
)

In [ ]:
## solution
px.line(
    ergebnisse.reset_index().query("Haushaltstyp == '2 Erwachsene, 2 Kinder'"),
    x="bruttolohn_m_hh",
    y=[
        "verfügbares_einkommen_kiz_und_wg",
        "verfügbares_einkommen_bürgergeld",
        "verfügbares_einkommen_vorrang",
    ],
    title="2 Erwachsene, 2 Kinder",
)

## Aufgabe 3: Pentabilities (10 Minuten)

Bitte bewerten Sie die Beiträge zur Gruppenarbeit in der Pentabilities-App. Bitte
vergeben Sie über alle 5 Dimensionen hinweg Punkte für die Verhaltensweisen, welche Sie
heute beobachten konnten. Denken Sie bitte daran, sowohl die Beiträge der
Kommiliton:innen Ihrer Gruppe als auch Ihre eigenen zu bewerten.

## Aufgabe 4: Friktionen der Inanspruchnahme (25 Minuten)

In dieser Aufgabe ignorieren wir, dass das Jobcenter eine Vorrangprüfung zwischen
Kinderzuschlag/Wohngeld einerseits und Bürgergeld andererseits vornimmt. Stattdessen
nehmen wir an, dass sich Haushalte für den ein oder anderen Transfer entscheiden können.

Diskutieren Sie anhand der Grafiken für was sich welche Familie ab welchem Einkommen
entscheiden würde und warum?

Beziehen Sie insbesondere die möglichen Friktionen der Inanspruchnahme der verschiedenen
Sozialleistungen mit ein.

!! solution

**Potentielle Antworten**

mögliche Friktionen:

- Transaktionskosten

- fehlende Information und hohe Komplexität

- Stigma

Transaktionskosten: 
- Sehr hoch bei Kinderzuschlag/Wohngeld weil muss bei 2 verschiedenen Behörden beantragt werden. 
- Insbesondere bei fluktuierendem Einkommen ein Riesenproblem.
- Außerdem muss Kinderzuschlag alle 6 Monate beantragt werden.
- Jedoch bei Bürgergeld -> regulärer Jobcenter Besuch verpflichtend.

-> Bevorzuge Bürgergeld

Fehlende Information:
- Wenige Leute wissen Bescheid über Kinderzuschlag.
- Berechnung von Wohngeld sehr komplex.
- Bürgergeld ist bekannter und zum Teil transparenter. 

-> Bevorzuge Bürgergeld.

Stigma
- Möglicherweise höher bei Bürgegeld Bezug.
- Eventuell seit Einführung des Bürgergelds weniger hoch als noch bei Hartz 4.

-> Bevorzuge Kinderzuschlag, Wohngeld selbst bei gleich hoher oder marginal geringerem Transfer.

Was genau gewählt wird hängt außerdem davon ab was davon zu gewinnen ist, also vom Einkommen, Miete, Haushaltstyp.


**Ausblick auf übernächste Woche:** Äquivalenzskalen, Vergleichbarkeit über unterschiedliche Haushalte hinweg.
